In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/WEB SCRAPING 101/job_description_only_after_cleaning.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18041 entries, 0 to 18040
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job_description     18041 non-null  object
 1   lower_description   18041 non-null  object
 2   word_tokenized      18041 non-null  object
 3   sentence_tokenized  18041 non-null  object
 4   clean_stopword      18041 non-null  object
 5   clean_lemmed        18041 non-null  object
dtypes: object(6)
memory usage: 845.8+ KB


In [ ]:
df2.drop_duplicates('lower_description', inplace= True)
df2.shape

# POS

In [ ]:
def pos_series(keyword):
    '''categorizes after tokenizing words with POS tags'''
    tokens = nltk.word_tokenize(keyword)
    tagged = nltk.pos_tag(tokens)
    return tagged

In [ ]:
pos_tagged_arrs = df2.lower_description.apply(pos_series)

In [ ]:
pos_tagged = []
for row in pos_tagged_arrs.values:
    for element in row:
        pos_tagged.append(element)

In [ ]:
pos_df = pd.DataFrame(pos_tagged, columns = ('word','POS'))

# remove unnecessary char
char_removal = [',', '.', ':', '#', '$', '\'\'', '``', '(', ')']
drop_indices = (pos_df.loc[pos_df.POS.isin(char_removal)].index)
pos_df.drop(drop_indices, inplace = True)

# Noun Prase #1

In [ ]:
# defining grammer within the text using reg expressions
# optional determinate, any number of adjectives, a noun, noun plural, proper noun with additionals following
grammar1 = ('''Noun Phrases: {<DT>?<JJ>*<NN|NNS|NNP>+}''')
chunkParser = nltk.RegexpParser(grammar1)
tree1 = chunkParser.parse(pos_tagged)

In [ ]:
# typical noun phrase pattern to be pickled for later analyses
g1_chunks = []
for subtree in tree1.subtrees(filter=lambda t: t.label() == 'Noun Phrases'):
    # print(subtree)
    g1_chunks.append(subtree)

In [ ]:
with open('/content/drive/My Drive/chunks_1.pickle', 'wb') as fp1:
    pickle.dump(g1_chunks, fp1)

# Noun Phrase #2

In [ ]:
# Noun phrase variation
# preposition maybe, any number of adjective or nouns, any plural nouns or singular nouns
grammar2 = ('''NP2: {<IN>?<JJ|NN>*<NNS|NN>} ''')
chunkParser = nltk.RegexpParser(grammar2)
tree2 = chunkParser.parse(pos_tagged)

In [ ]:
# variation of a noun phrase pattern to be pickled for later analyses
g2_chunks = []
for subtree in tree2.subtrees(filter=lambda t: t.label() == 'NP2'):
    # print(subtree)
    g2_chunks.append(subtree)

In [ ]:
with open('/content/drive/My Drive/chunks_2.pickle', 'wb') as fp2:
    pickle.dump(g2_chunks , fp2)

Verb-Noun Pattern #3

In [ ]:
# any sort of verb followed by any number of nouns
grammar3 = ('''
    VS: {<VBG|VBZ|VBP|VBD|VB|VBN><NNS|NN>*}
    ''')
chunkParser = nltk.RegexpParser(grammar3)
tree3 = chunkParser.parse(pos_tagged)

In [ ]:
# verb-noun pattern to be pickled for later analyses
g3_chunks = []
for subtree in tree3.subtrees(filter=lambda t: t.label() == 'VS'):
    # print(subtree)
    g3_chunks.append(subtree)

In [ ]:
with open('/content/drive/My Drive/chunks_3.pickle', 'wb') as fp3:
    pickle.dump(g3_chunks, fp3)


# <noun, noun*> Pattern #4

In [ ]:
# any number of a singular or plural noun followed by a comma followed by the same noun, noun, noun pattern
grammar4 = ('''
    Commas: {<NN|NNS>*<,><NN|NNS>*<,><NN|NNS>*}
    ''')
chunkParser = nltk.RegexpParser(grammar4)
tree4 = chunkParser.parse(pos_tagged)

In [ ]:
# common pattern of listing skills to be pickled for later analyses
g4_chunks = []
for subtree in tree4.subtrees(filter=lambda t: t.label() == 'Commas'):
    # print(subtree)
    g4_chunks.append(subtree)


In [ ]:
with open('/content/drive/My Drive/chunks_4.pickle', 'wb') as fp4:
    pickle.dump(g4_chunks, fp4)

## Load Chunk

In [ ]:
chunks1 = pickle.load( open('/content/drive/MyDrive/chunks_1.pickle', "rb" ) )
chunks2 = pickle.load( open('/content/drive/MyDrive/chunks_2.pickle', "rb" ) )
chunks3 = pickle.load( open('/content/drive/MyDrive/chunks_3.pickle', "rb" ) )
chunks4 = pickle.load( open('/content/drive/MyDrive/chunks_4.pickle', "rb" ) )

In [ ]:
chunks1

In [ ]:
print('Length:', len(chunks1), 'Sample Size:', len(chunks1) * .10)
print('Length:', len(chunks2), 'Sample Size:', len(chunks2) * .10)
print('Length:', len(chunks3), 'Sample Size:', len(chunks3) * .10)
print('Length:', len(chunks4), 'Sample Size:', len(chunks4) * .10)

Length: 1418223 Sample Size: 141822.30000000002
Length: 1453992 Sample Size: 145399.2
Length: 707018 Sample Size: 70701.8
Length: 60326 Sample Size: 6032.6
